# Digital Twin v5.0: The Elite Stacking Ensemble

**Author:** Kian Mansouri Jamshidi
**Project Director:** Kian Mansouri Jamshidi
**Date:** 2025-09-27

## Objective
This is the final and most refined architecture of Sprint 5. We will construct an 'Elite' hierarchical ensemble. This network focuses only on our two best-performing and most diverse base models (v2.0 and v2.3). These feed into a dual-meta-AI layer, which is then unified by a final 'Superior AI'. This tests the hypothesis that a more focused ensemble can outperform a more complex but noisy one.

### 1. Imports and Data Preparation

In [1]:
import pandas as pd
import numpy as np
import glob
import joblib
from pathlib import Path

from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import r2_score
from sklearn.linear_model import RidgeCV
import lightgbm as lgb

PROJECT_ROOT = Path('.').resolve().parent
TELEMETRY_DIR = PROJECT_ROOT / 'data' / 'telemetry_v2'
ARTIFACT_DIR = PROJECT_ROOT / 'artifacts' / 'phase2'
df_list = [pd.read_parquet(file) for file in glob.glob(str(TELEMETRY_DIR / "*.parquet"))]
df = pd.concat(df_list, ignore_index=True).sort_values(by='timestamp').reset_index(drop=True)
if 'cpu_temp_celsius_avg' in df.columns:
    df = df.drop('cpu_temp_celsius_avg', axis=1)
print("Data loaded.")

# --- Load The Two Elite Base Models (Layer 1) ---
model_v2_0 = joblib.load(ARTIFACT_DIR / 'digital_twin_v2.0.joblib')
model_v2_3 = joblib.load(ARTIFACT_DIR / 'digital_twin_v2.3_deep.joblib')
print("Elite base models v2.0 and v2.3 loaded successfully.")

Data loaded.
Elite base models v2.0 and v2.3 loaded successfully.


### 2. Feature Generation and Alignment

In [2]:
def create_v2_0_features(df):
    df_f=df.copy();workload_dummies=pd.get_dummies(df_f['workload_type'],prefix='workload');df_f=pd.concat([df_f,workload_dummies],axis=1);df_f['overall_util_rolling_mean']=df_f['cpu_util_overall'].rolling(window=10).mean();df_f['overall_util_rolling_std']=df_f['cpu_util_overall'].rolling(window=10).std();other_cores=[c for c in df.columns if 'cpu_util_core' in c and c != 'cpu_util_core_0'];
    for i in range(1,6):
        df_f[f'overall_util_lag_{i}']=df_f['cpu_util_overall'].shift(i)
        for core in other_cores:df_f[f'{core}_lag_{i}']=df_f[core].shift(i)
    return df_f.drop('workload_type',axis=1).dropna().reset_index(drop=True)

def create_v2_3_features(df):
    df_f=df.copy();workload_dummies=pd.get_dummies(df_f['workload_type'],prefix='workload');df_f=pd.concat([df_f,workload_dummies],axis=1);df_f['overall_util_rolling_mean']=df_f['cpu_util_overall'].rolling(window=30).mean();df_f['overall_util_rolling_std']=df_f['cpu_util_overall'].rolling(window=30).std();other_cores=[c for c in df.columns if 'cpu_util_core' in c and c != 'cpu_util_core_0'];
    for i in range(1,21):
        df_f[f'overall_util_lag_{i}']=df_f['cpu_util_overall'].shift(i)
        if i<=10: 
            for core in other_cores:df_f[f'{core}_lag_{i}']=df_f[core].shift(i)
    return df_f.drop('workload_type',axis=1).dropna().reset_index(drop=True)

df_v2_0=create_v2_0_features(df);df_v2_3=create_v2_3_features(df);target='cpu_util_core_0';common_indices=df_v2_0.index.intersection(df_v2_3.index);df_aligned_v2_0=df_v2_0.loc[common_indices];df_aligned_v2_3=df_v2_3.loc[common_indices];
features_v2_0_cols=[c for c in df_aligned_v2_0.columns if ('cpu_util' in c and c!=target) or 'workload_' in c];features_v2_3_cols=[c for c in df_aligned_v2_3.columns if ('cpu_util' in c and c!=target) or 'workload_' in c];
X_v2_0=df_aligned_v2_0[features_v2_0_cols];X_v2_3=df_aligned_v2_3[features_v2_3_cols];y=df_aligned_v2_0[target];
print(f"Data aligned. Final dataset size: {len(y)} rows.")

Data aligned. Final dataset size: 6899 rows.


### 3. Hierarchical Training and Evaluation

In [3]:
indices = np.arange(len(y))
train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)

print("--- Stage 1: Generating Layer 1 Predictions ---")
layer1_preds_train_v2_0 = cross_val_predict(model_v2_0, X_v2_0.iloc[train_indices], y.iloc[train_indices], cv=5, n_jobs=-1)
layer1_preds_train_v2_3 = cross_val_predict(model_v2_3, X_v2_3.iloc[train_indices], y.iloc[train_indices], cv=5, n_jobs=-1)
X_meta_train_L2 = pd.DataFrame({'pred_v2_0': layer1_preds_train_v2_0, 'pred_v2_3': layer1_preds_train_v2_3})
y_train_L2 = y.iloc[train_indices]

print("--- Stage 2: Training Layer 2 Meta-Models ---")
meta_model_X = RidgeCV()
meta_model_Y = lgb.LGBMRegressor(random_state=42, n_jobs=-1)
layer2_preds_train_X = cross_val_predict(meta_model_X, X_meta_train_L2, y_train_L2, cv=5, n_jobs=-1)
layer2_preds_train_Y = cross_val_predict(meta_model_Y, X_meta_train_L2, y_train_L2, cv=5, n_jobs=-1)
X_meta_train_L3 = pd.DataFrame({'pred_meta_X': layer2_preds_train_X, 'pred_meta_Y': layer2_preds_train_Y})
y_train_L3 = y_train_L2

print("--- Stage 3: Training Layer 3 Superior AI ---")
superior_model = lgb.LGBMRegressor(**model_v2_3.get_params())
superior_model.fit(X_meta_train_L3, y_train_L3)

print("--- Evaluating on hold-out test set ---")
layer1_preds_test_v2_0=model_v2_0.predict(X_v2_0.iloc[test_indices]);layer1_preds_test_v2_3=model_v2_3.predict(X_v2_3.iloc[test_indices]);
X_meta_test_L2=pd.DataFrame({'pred_v2_0':layer1_preds_test_v2_0,'pred_v2_3':layer1_preds_test_v2_3})
meta_model_X.fit(X_meta_train_L2,y_train_L2);meta_model_Y.fit(X_meta_train_L2,y_train_L2);
layer2_preds_test_X=meta_model_X.predict(X_meta_test_L2);layer2_preds_test_Y=meta_model_Y.predict(X_meta_test_L2);
X_meta_test_L3=pd.DataFrame({'pred_meta_X':layer2_preds_test_X,'pred_meta_Y':layer2_preds_test_Y})
final_predictions=superior_model.predict(X_meta_test_L3)
y_test=y.iloc[test_indices]
r2 = r2_score(y_test, final_predictions)

print(f"\n--- Final Elite Ensemble Performance ---")
print(f"R-squared (R²): {r2:.4f}")

if r2 >= 0.85:
    print("\nMISSION ACCOMPLISHED: The Elite Ensemble has broken the 85% barrier!")
elif r2 > 0.7436:
    print("\nULTIMATE BREAKTHROUGH: The Elite Ensemble is the new state-of-the-art.")
else:
    print("\nLIMIT REACHED: The Elite Ensemble did not improve performance. The V2.5 Stacking model remains the champion.")

--- Stage 1: Generating Layer 1 Predictions ---
--- Stage 2: Training Layer 2 Meta-Models ---
--- Stage 3: Training Layer 3 Superior AI ---
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000107 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 5519, number of used features: 2
--- Evaluating on hold-out test set ---
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002056 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 5519, number of used features: 2
[LightGBM] [Info] Start training from score 4.857456

--- Final Elite Ensemble Performance ---
R-squared (R²): 0.6716

LIMIT REACHED: The Elite Ensemble did not improve performance. The V2.5 Stacking model remains the champion.
